# AJAX방식으로 구현하기

# 1. 수정후, 기존의 page번호가 나오게하기
> 수정한 후
- 두번째 페이지에서 수정했으면, 수정한 후, 두번째 페이지로 나와야함
- 되돌아갈 페이지에 대한 정보가 있어야 함.


> 수정한다고 한 이후, 목록으로 다시 가기
- 리스트로 가야 하는데 page1로 가면 안되니까
- 이전 페이지로 가게 만들어야 함.
>> 이전페이지로 가기: 
- 자바스크립트에서 history.back하면 가능
- 그러나 수정된 사항은 반영안됨

> add할 때
- ordering이 안되어 있으므로 최근거부터 나오게 하기
- 상관없음.


> 수정을 만들 때 rink에 page번호를 같이 넣어줘야 함.
- url 체계에 주는 것이 명확함

## 1.1. url체계를 바꾸기

### 비동기식으로 데이터 수정을 클라이언트에서 처리
> 데이터가 없어지면
- redirect하기 때문에 서버가 다시 데이터를 계산해서 다시 렌더링해서 보내야 함.
- Ajax 개념 사용하기


> Ajax개념
- 페이지를 갱신하지 않고,
- 페이지방식으로 하면 업데이트 된 것만 지울 수 없음.
- 지워주세요 요청하면 화면은 그대로 두고
- 특정 url 요청(이 데이터 지워주세요)
- 옛날 데이터를 갖고 있는 상태에서 html에서만 지우기

> 다시 list를 생성하는 작업
- 데이터양도 많고
- 사용자도 많으므로
- view 화면을 한번 갱신하는 것은 힘든작업
- 서버의 가용성을 높일 수 잇음.

> 프론트엔드에서
- 데이터를 처리
- 클라이언트 사이드 처리

# 2. 페이지 번호 누르지 않아도 스크롤로 다음 페이지 가져오도록 하기
> 데이터 바인딩
+ 누르면
- 기존의 데이터 아래 새로운 데이터 아래 같이 보여주기
- 서버에 부담(네트워크 트래픽 증가)
- -> json형태로 갖고와서 클라이언트에서 원래 데이터에 붙이기

## 2.1. get 방식
### 2.1.1 지워달라는 요청, 데이터를 보내달라는 요청을 따로 처리

- ajaxdel(): 지우는 함수
- ajaxget(): 데이터 보내는 함수(page번호를 요청했을 때 데이터를 page 단위로 보내주기)

In [1]:
# urls.py
'''
from django.urls import path
from . import views 
from django.shortcuts import redirect

urlpatterns = [
    path('', views.page),
    path('ajaxdel', views.ajaxdel),
    path('ajaxget', views.ajaxget),
    path('<category>/<int:pk>/<mode>', views.BoardView.as_view(), name = 'myboard'),
    ]

'''

"\nfrom django.urls import path\nfrom . import views \nfrom django.shortcuts import redirect\n\nurlpatterns = [\n    path('', views.page),\n    path('ajaxdel', views.ajaxdel),\n    path('ajaxget', views.ajaxget),\n    path('<category>/<int:pk>/<mode>', views.BoardView.as_view(), name = 'myboard'),\n    ]\n\n"

In [ ]:
# views.py
def ajaxdel(request):
    pk = request.GET.get('pk')
    board = models.Board.objects.all().get(pk = pk)
    board.delete()
    return JsonResponse({'error': '0'})


# 특정 페이지의 모든 데이터 
def ajaxget(request):
    page = request.GET.get('page', 1)
    datas = models.Board.objects.all().filter(category = 'common')
    
    # page개념이 아니므로 슬라이싱으로 해도 상관없음.
    # page = int(page)
    # subs = datas[(page-1)*3: (page*3)]
    p = Paginator(datas, 3)
    subs = p.page(page)

    datas = {'datas': [{'pk': data.pk, 'title': data.title, 'cnt': data.cnt} for data in subs]}
    return JsonResponse(datas)

> 실행
- http://127.0.0.1:8000/myboard/ajaxdel?pk=1
- http://127.0.0.1:8000/myboard/ajaxget?page=1

In [ ]:
# static > jquery.html ( = jquerytest.html)
'''
head>
    <meta charset="UTF-8">
  </head>
  
  
  <script src="http://code.jquery.com/jquery-1.11.3.min.js"></script>
  <script src="http://code.jquery.com/jquery-migrate-1.2.1.min.js"></script>
  
  <button id="btnMore">More</button>
  
  
  
  <div id="view">
    <div id="item">
     <a href=list?id=4> <span id=title>제목~~~~~~~~</span> </a> 
       조회수<span id=cnt> 0 </span>
     <br>
    <div> 
  </div>   
  
  <script>
     //$("#view").append("<a href=fefef>데이터2</a><br>")
     //$("#view").append("<a href=fefef>데이터2</a><br>")
     //$("#title").html("제목1")
     //$("#cnt").text("10")
     
     $("#item").hide();  
     
     var page = 1
     
     function getPage(page) {
          //json = {"datas": [{"pk": 1, "title": "자동차 검출44343", "cnt": 5}, {"pk": 2, "title": "32321321", "cnt": 0}, {"pk": 3, "title": "32312323123", "cnt": 0}]}
          
         $.get("/myboard/ajaxget", {page:page}, function(json) {	   
              console.log(JSON.stringify(json));
              
              if ( json.datas.length == 0) alert("마지막페이지 입니다.");
              for (i = 0; i < json.datas.length; i++) {		
                  item = $("#item").clone();
                  item.show();
                  $("#title", item).html(json.datas[i].title);
                  $("#cnt", item).text(json.datas[i].cnt);      		
                  $("#view").append(item);   					
              }   
          });		
     }    
     $("#btnMore").click( function() {  
              page++;
              getPage(page);
     });
     
     getPage(page);   
     
  
  
  
  </script>
'''

## 2.1 url체계를 바꾸기 어려운 경우
- url에 있는 파라미터 정보를 넘겨주기

> post방식에서 action을 해야 하느네
- action값이 자기 자긴이름이므로
<input type = 'hidden' name = "page" value = {{data.pagenum}}>으로 보이지 않지만 값을 갖고 다니면서 보녀줘야 함.

## 2.3. 쿠키 정보도 있으나 잘 사용하지 않음.
> 또 다른 방법: 쿠키 데이터에 기록할 수도 있음.
- 보안상의 문제로 
- 쿠키 정보가 없으면 동일한 url임에도 같은 화면이 나오지 않으므로

# 3. 삭제

### 3.1.삭제 버튼 만들기

#### 페이지 바꾸기(동기식)

In [ ]:
'''
  <div id="view">
    <div id="item">
     <a href=list?id=4> <span id=title>제목~~~~~~~~</span> </a> 
       조회수<span id=cnt> 0 </span>
       <a href="/myboard/ajaxdel?pk = 1">지우기</a>
     <br>
    <div> 
  </div>   
'''

### 비동기식으로

####  자바 스크립트 함수, 속성값 변경 연습

In [2]:
'''
 <div id="view">
    <div id="item">
     <a href=list?id=4> <span id=title>제목~~~~~~~~</span> </a> 
       조회수<span id=cnt> 0 </span>
       <!-- 자바스크립트 함수 호출 -> 페이지가 넘어가지 않음.-->
       <a href="javascript: deleteBoard(1)" id = del>지우기</a>
     <br>
    <div> 
  </div>   
  
  <script>
    //  tag의 속성값 바꾸기 -> 6이 나와야함.
    $('#del').attr('href', 'javascript:deleteBoard(6)');
    function deleteBoard(pk){
        alert(pk);
    }


'''

'\n <div id="view">\n    <div id="item">\n     <a href=list?id=4> <span id=title>제목~~~~~~~~</span> </a> \n       조회수<span id=cnt> 0 </span>\n       <!-- 자바스크립트 함수 호출 -> 페이지가 넘어가지 않음.-->\n       <a href="javascript: deleteBoard(1)" id = del>지우기</a>\n     <br>\n    <div> \n  </div>   \n  \n  <script>\n    //  tag의 속성값 바꾸기 -> 6이 나와야함.\n    $(\'#del\').attr(\'href\', \'javascript:deleteBoard(6)\');\n    function deleteBoard(pk){\n        alert(pk);\n    }\n\n\n'

#### pk값으로 자바스크립트가 호출될 수 있도록 하기

> item을 지우기
- item을 지워야 함.
- item을 만들 때 고유 아이디를 만들어서 attribute 값을 바꾸기

In [ ]:
'''
<button id="btnMore">More</button>
  
  
  
  <div id="view">
    <div id="item">
     <a href=list?id=4> <span id=title>제목~~~~~~~~</span> </a> 
       조회수<span id=cnt> 0 </span>
       <!-- 자바스크립트 함수 호출 -> 페이지가 넘어가지 않음.-->
       <a href="javascript: deleteBoard()" id = 'del'>지우기</a>
     <br>
    <div> 
  </div>   
  

  <script>
    //  tag의 속성값 바꾸기 -> 6이 나와야함.
    // $('#del').attr('href', 'javascript:deleteBoard(6)');

    /////// 지우기
    function deleteBoard(pk){
        // alert(pk);
        $.get('/myboard/ajaxdel', {pk: pk}, function(json){
                // alert(json.error);
                // db에서 제거한 후, 클라이어트로 넘어옴
                $('#item_'+pk).remove();
        });
        //get 함수 이후에 지우면 db에서 안지워지고, 클라이언트에서만 지워짐. 
        // $('#item_'+pk).remove();

    }
    
    //////// add -> page 추가 
    $("#item").hide();  
   
   var page = 1;
   
   function getPage(page) {
		
	   $.get("/myboard/ajaxget", {page:page}, function(json) {	   
            console.log(JSON.stringify(json));
            
            if (json.datas.length == 0) alert('마지막입니다');
			for (i = 0; i < json.datas.length; i++) {		
                item = $("#item").clone();
                // # item의 속성을 pk값으로 바꾸기
                item.attr('id', 'item_' + json.datas[i].pk)
				item.show();
                $("#title", item).html(json.datas[i].title);
                $('#del', item).attr('href', 'javascript:deleteBoard('+json.datas[i].pk+")");
				$("#cnt", item).text(json.datas[i].cnt);      		
				$("#view").append(item);   					
			}   
		});		
   }    

   $("#btnMore").click( function() {  
			page++;
			getPage(page);
   });
   
   getPage(page);   
   



</script>
'''